In [1]:
'''
Sentiment analysis, a common Natural Language Processing method, was applied to big data of Amazon reviews. 
It will process the large-scale datasets using Apache Spark's RDD to partition and distribute.
The pre-processing on the big data will be done using PySpark, and the in-depth processing will be done in Python.
Like real world situations, we will take the best of both worlds.
'''

In [2]:
# loading to train and test RDDs, creating union of train and test
train = sc.textFile("/FileStore/tables/train_ft-bfd00.txt", 2) 
test = sc.textFile("/FileStore/tables/test_ft-41af5.txt", 2)
df = train.union(test)

In [3]:
# first 2 train set rows
train.take(2)

Out[ 3 ]: 
['__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^',
 "__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."]

In [4]:
# first 2 test set rows
test.take(2)

Out[ 21 ]: 
['__label__2 Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I\'m in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life\'s hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"',
 "__label__2 One of the best game music soundtracks - for a game I didn't really play: Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of those kinds of songs in my other video game soundtracks. I must admit that one of the songs (Life-A Distant Promise) has brought tears to my eyes on many occasions.My one complaint about this soundtrack is that they use guitar fretting effects in many of the songs, which I find distracting. But even if those weren't included I would still consider the collection worth it."]

In [5]:
'''
remove \
('__level__1 or __level__2  review title : review sentences'  ,  'train1' or 'test1'
1 indicates bad, 2 indicates good
CAPS for emphasizing
'''

In [6]:
import time
time_start = time.time()

# cleaning to get label numbers and reviews 
df = df.map(lambda x: x.split('__label__')[1].split(" ", 1))
print("Size of dataset: ", df.count())
# counting the time
print('Split done! Time elapsed: {} seconds'.format(time.time()-time_start))
df.take(2)

Size of dataset: 4000000
Split done! Time elapsed: 23.306551933288574 seconds
 Out[ 4 ]: 
[['2',
 'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'],
 ['2',
 "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."]]

In [7]:
# convert PipelineRDD to DataFrame
df1 = df.toDF(["Label","Review"])
df1.describe()
df1.show(2)

+-----+--------------------+
Label| Review|
+-----+--------------------+
 2|Stuning even for ...|
 2|The best soundtra...|
+-----+--------------------+
only showing top 2 rows

In [8]:
df1 = df1.dropna()

In [9]:
type(df1)

Out[ 6 ]: pyspark.sql.dataframe.DataFrame

In [10]:
# label 2 = good , label 1 = bad
if df1["Label"]=="2":
  df1["Label"].replace("2", "good")
else:
  df1["Label"].replace("1", "bad")

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-1532425816074300> in <module> () 
 1 # label 2 = good , label 1 = bad 
 ----> 2 if df1 [ "Label" ] == "2" : 
 3 df1 [ "Label" ] . replace ( "2" , "good" ) 
 4 else : 
 5 df1 [ "Label" ] . replace ( "1" , "bad" ) 

 /databricks/spark/python/pyspark/sql/column.py in __nonzero__ (self) 
 680 
 681 def __nonzero__ ( self ) : 
 --> 682 raise ValueError("Cannot convert column into bool: please use '&' for 'and', '|' for 'or', "
 683 "'~' for 'not' when building DataFrame boolean expressions.")
 684 __bool__ = __nonzero__ 

 ValueError : Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [11]:
from pyspark.sql.functions import lit
new_df = df1.withColumn('Label', lit("good"))

In [12]:
sentiment="good" if(df1[0]=="2") else "bad"

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-1532425816074299> in <module> () 
 ----> 1 sentiment = "good" if ( df1 [ 0 ] == "2" ) else "bad" 
 2 sentence = temp [ 1 : len ( temp ) ] 

 /databricks/spark/python/pyspark/sql/column.py in __nonzero__ (self) 
 680 
 681 def __nonzero__ ( self ) : 
 --> 682 raise ValueError("Cannot convert column into bool: please use '&' for 'and', '|' for 'or', "
 683 "'~' for 'not' when building DataFrame boolean expressions.")
 684 __bool__ = __nonzero__ 

 ValueError : Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [13]:
# compare the 2 labels
import seaborn as sns
sns.countplot(df1['Label'])

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1742976175039855> in <module> () 
 1 # compare the 2 labels 
 2 import seaborn as sns 
 ----> 3 sns . countplot ( df1 [ 'Label' ] ) 

 /databricks/python/lib/python3.5/site-packages/seaborn/categorical.py in countplot (x, y, hue, data, order, hue_order, orient, color, palette, saturation, ax, **kwargs) 
 3256 estimator , ci , n_boot , units , 
 3257 orient , color , palette , saturation , 
 -> 3258 errcolor)
 3259 
 3260 plotter . value_label = "count" 

 /databricks/python/lib/python3.5/site-packages/seaborn/categorical.py in __init__ (self, x, y, hue, data, order, hue_order, estimator, ci, n_boot, units, orient, color, palette, saturation, errcolor, errwidth, capsize) 
 1541 """Initialize the plotter.""" 
 1542 self.establish_variables(x, y, hue, data, orient,
 -> 1543 order, hue_order, units)
 1544 self . establish_colors ( color , palette , saturation ) 
 1545 self . estimate_statistic ( estimator , ci , n_boot ) 

 /databricks/python/lib/python3.5/site-packages/seaborn/categorical.py in establish_variables (self, x, y, hue, data, orient, order, hue_order, units) 
 197 
 198 # Get the order on the categorical axis 
 --> 199 group_names = categorical_order ( groups , order ) 
 200 
 201 # Group the numeric data 

 /databricks/python/lib/python3.5/site-packages/seaborn/utils.py in categorical_order (values, order) 
 531 except ( ValueError , TypeError ) : 
 532 order = order 
 --> 533 order = filter ( pd . notnull , order ) 
 534 return list ( order ) 
 535 

 /databricks/spark/python/pyspark/sql/column.py in __iter__ (self) 
 342 
 343 def __iter__ ( self ) : 
 --> 344 raise TypeError ( "Column is not iterable" ) 
 345 
 346 # string methods 

 TypeError : Column is not iterable

In [14]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml import Pipeline

# Tokenize the review 
tokenizer = Tokenizer(inputCol="Review", outputCol="review_words")
wordsDF = tokenizer.transform(df1)
wordsDF.show(2)

# Remove stop words
remover = StopWordsRemover(inputCol="review_words", outputCol="filtered")
wordsDF2 = remover.transform(wordsDF)
wordsDF2.show(2)

# Convert to TF words vector
hashingTF = HashingTF(inputCol="filtered", outputCol="TF")
wordsDF3 = hashingTF.transform(wordsDF2)
wordsDF3.show(2)

# Convert to IDF words vector, ensure to name the features as 'features'
idf = IDF(inputCol="TF", outputCol="features")
idfModel = idf.fit(wordsDF3)
wordsDF4 = idfModel.transform(wordsDF3)
wordsDF4.show(2)

# HashingTF in SparkML can't normalize term frequency with the total number of words in each document
for features_label in wordsDF3.select("TF", "label").take(1):
    print(features_label)

# IDF features
for features_label in wordsDF4.select("features", "label").take(1):
    print("\n",features_label)   

+-----+--------------------+--------------------+
Label| Review| review_words|
+-----+--------------------+--------------------+
 2|Stuning even for ...|[stuning, even, f...|
 2|The best soundtra...|[the, best, sound...|
+-----+--------------------+--------------------+
only showing top 2 rows

+-----+--------------------+--------------------+--------------------+
Label| Review| review_words| filtered|
+-----+--------------------+--------------------+--------------------+
 2|Stuning even for ...|[stuning, even, f...|[stuning, even, n...|
 2|The best soundtra...|[the, best, sound...|[best, soundtrack...|
+-----+--------------------+--------------------+--------------------+
only showing top 2 rows

+-----+--------------------+--------------------+--------------------+--------------------+
Label| Review| review_words| filtered| TF|
+-----+--------------------+--------------------+--------------------+--------------------+
 2|Stuning even for ...|[stuning, even, f...|[stuning, even, n...|(262144,[9129,133...|
 2|The best soundtra...|[the, best, sound...|[best, soundtrack...|(262144,[2991,132...|
+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
Label| Review| review_words| filtered| TF| features|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 2|Stuning even for ...|[stuning, even, f...|[stuning, even, n...|(262144,[9129,133...|(262144,[9129,133...|
 2|The best soundtra...|[the, best, sound...|[best, soundtrack...|(262144,[2991,132...|(262144,[2991,132...|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows

Row(TF=SparseVector(262144, {9129: 1.0, 13381: 1.0, 20069: 1.0, 26471: 1.0, 32826: 1.0, 32957: 1.0, 41006: 1.0, 41704: 1.0, 50134: 1.0, 62790: 1.0, 68203: 1.0, 72709: 1.0, 87566: 2.0, 93398: 1.0, 96316: 1.0, 98065: 1.0, 102573: 1.0, 112623: 1.0, 114686: 1.0, 138193: 1.0, 138895: 2.0, 145080: 1.0, 149784: 1.0, 152471: 1.0, 166027: 1.0, 170317: 1.0, 170688: 1.0, 174966: 2.0, 178465: 1.0, 186925: 1.0, 188123: 1.0, 193400: 1.0, 203802: 1.0, 207954: 1.0, 209326: 1.0, 217825: 2.0, 235131: 1.0, 237887: 1.0}), label='2')

 Row(features=SparseVector(262144, {9129: 3.7191, 13381: 3.3395, 20069: 7.8474, 26471: 8.5698, 32826: 10.2965, 32957: 4.4766, 41006: 9.2382, 41704: 4.8905, 50134: 3.9394, 62790: 5.2688, 68203: 8.1933, 72709: 4.5903, 87566: 14.3915, 93398: 6.7889, 96316: 6.13, 98065: 8.531, 102573: 10.4227, 112623: 7.089, 114686: 2.8651, 138193: 3.2664, 138895: 7.455, 145080: 8.1405, 149784: 11.8006, 152471: 7.3906, 166027: 2.5201, 170317: 11.4406, 170688: 4.3974, 174966: 4.2566, 178465: 4.8931, 186925: 2.4811, 188123: 5.3229, 193400: 8.1915, 203802: 2.8574, 207954: 6.1171, 209326: 6.6664, 217825: 8.6082, 235131: 9.3102, 237887: 7.7079}), label='2')

In [15]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Split data into training and testing set 
(training, test) = df1.randomSplit([0.7, 0.3])

# Create a logistic regression instance
lr = LogisticRegression(maxIter=10)

# Use a pipeline to chain all transformers and estimators
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idfModel, lr])

# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 50]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3) 

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)
selected = prediction.select("Review", "Label", "probability", "prediction").take(5)
for row in selected:
    print(row)

# Evaluate result with ROC
evaluator = BinaryClassificationEvaluator(
    labelCol="Label", metricName="areaUnderROC")
ROC = evaluator.evaluate(prediction)
ROC

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-3451440845433279> in <module> () 
 25 
 26 # Run cross-validation, and choose the best set of parameters. 
 ---> 27 cvModel = crossval . fit ( training ) 
 28 
 29 # Make predictions on test documents. cvModel uses the best model found (lrModel). 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 302 
 303 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 304 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 305 metrics [ j ] += ( metric / nFolds ) 
 306 if collectSubModelsParam : 

 /usr/lib/python3.5/multiprocessing/pool.py in next (self, timeout) 
 693 if success : 
 694 return value 
 --> 695 raise value 
 696 
 697 __next__ = next # XXX 

 /usr/lib/python3.5/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 117 job , i , func , args , kwds = task 
 118 try : 
 --> 119 result = ( True , func ( * args , ** kwds ) ) 
 120 except Exception as e : 
 121 if wrap_exception : 

 /databricks/spark/python/pyspark/ml/tuning.py in <lambda> (f) 
 302 
 303 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 304 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 305 metrics [ j ] += ( metric / nFolds ) 
 306 if collectSubModelsParam : 

 /databricks/spark/python/pyspark/ml/tuning.py in singleTask () 
 50 
 51 def singleTask ( ) : 
 ---> 52 index , model = next ( modelIter ) 
 53 metric = eva . evaluate ( model . transform ( validation , epm [ index ] ) ) 
 54 return index , metric , model if collectSubModel else None 

 /databricks/spark/python/pyspark/ml/base.py in __next__ (self) 
 60 raise StopIteration ( "No models remaining." ) 
 61 self . counter += 1 
 ---> 62 return index , self . fitSingleModel ( index ) 
 63 
 64 def next ( self ) : 

 /databricks/spark/python/pyspark/ml/base.py in fitSingleModel (index) 
 104 
 105 def fitSingleModel ( index ) : 
 --> 106 return estimator . fit ( dataset , paramMaps [ index ] ) 
 107 
 108 return _FitMultipleIterator ( fitSingleModel , len ( paramMaps ) ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 128 elif isinstance ( params , dict ) : 
 129 if params : 
 --> 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 132 return self . _fit ( dataset ) 

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 --> 109 model = stage . fit ( dataset ) 
 110 transformers . append ( model ) 
 111 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
